In [1]:
import pandas as pd
import numpy as np
import random
import os
import torch
import sys
sys.path.append('../')
from lillian_modules import utils, simple_constants
from lillian_modules.simple_env import SimpleEnv
from stable_baselines3.dqn import DQN
# from modules.env import LupusEnv
from sklearn.model_selection import train_test_split
# from stable_baselines3 import DQN
from imblearn.under_sampling import ClusterCentroids, RandomUnderSampler
import warnings
warnings.filterwarnings('ignore')

ImportError: cannot import name 'simple_constants' from 'lillian_modules' (unknown location)

In [ ]:
SEED = simple_constants.SEED
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
torch.manual_seed(SEED)
torch.use_deterministic_algorithms(True)

In [ ]:
testing_df = pd.read_csv('../../RL-Agent-Diagnosis/lupus/data/missingness/0/testing_set.csv')
# testing_df = pd.read_csv('../../RL-Agent-Diagnosis/lupus/data/inconclusive_diagnosis/missingness/0/testing_set.csv')
X_test, y_test = testing_df.iloc[:, 0:-1], testing_df.iloc[:, -1]
X_test, y_test = np.array(X_test), np.array(y_test)
testing_df.head()

In [ ]:
model = utils.load_dqn3('../models/very_simple_models/negative_step_reward/unbalanced/11_03_23/23_ft_incorr_-10_corr_1/best_acc_model.zip')
# model = utils.load_dqn3('../models/inconclusive_diagnosis/04_03_23/23_ft_incorr_-1_corr_1/best_acc_model.zip')
# model = utils.load_dqn3('../models/very_simple_models/best_models/23_ft_20000000/best_acc_model.zip')

In [ ]:
# test_df = pd.read_csv('../../RL-Agent-Diagnosis/lupus/test_dfs/21_jan/dqn_test_df_13000000.csv')

In [ ]:
test_df = pd.DataFrame()
testing_env = SimpleEnv(X_test, y_test, random=False)
# testing_env = LupusEnv(X_test, y_test, random=False)
count = 0
try:
    while True:
        count += 1
        if count %(len(X_test)/5)==0:
            print(f'Count: {count}')
        obs, done = testing_env.reset(), False
        while not done:
            action, states = model.predict(obs, deterministic=True)
            obs, rew, done, info = testing_env.step(action)
            if done==True:
                test_df = test_df.append(info, ignore_index=True)
except StopIteration:
    print('Testing done ....')
test_df.head() 

In [ ]:
test_df.episode_length.min(), test_df.episode_length.max()

In [ ]:
success_rate, success_df = utils.success_rate(test_df)
success_rate

In [ ]:
acc, f1, roc_auc, precision, recall = utils.test(test_df['y_actual'], test_df['y_pred'])
acc, f1, roc_auc, precision, recall

In [ ]:
avg_length, avg_return = utils.get_avg_length_reward(test_df)
avg_length, avg_return

In [ ]:
# test_df.to_csv('../../RL-Agent-Diagnosis/lupus/test_dfs/negative_reward/penalty_on_positive.csv', index=False)

###### 